In [1]:
from util import *
import time

In [2]:
def load_file_chunk(filename, n_each, n_total):
    #n_each is the lines read for each time
    #n_total, the total lines read for all

    with open(filename, "r",encoding="UTF-8") as f:
        """
        load data
        token is split by "\t"
        """
        head = f.readline().strip().split("\t")
        lines = []

        i = 0
        total = 0
        for line in f:
            lines.append(line.strip().split("\t"))
            
            i+=1
            total+=1

            if i >= n_each:
                yield head, lines
                lines = []
                i = 0

            if total >= n_total:
                break

        yield head, lines

In [3]:
if __name__ == "__main__":
    start = time.time()
    read_file  = "kgtk_conceptnet.tsv"
    write_file = "./data/cn_predict.tsv"
    write_file_prob = "./data/cn_missing_prob.tsv"
    write_file_freq = "./data/cn_freq.tsv"
    n_each = 100
    n_total = 300
        
    
    #no synset label num
    count1 = 0
    #no synset triple num
    count2 = 0
    # current total counted lines
    total_count = 0
    
    # initilization of file
    with open(write_file, "w", newline='',encoding="UTF-8") as f:
        new_head = ['node1;label','relation','node2;label','node1','node2']
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(new_head)
        
    # initilization of file
    with open(write_file_prob, "w", newline='',encoding="UTF-8") as f:
        f.write("prob1: prob of missing synsets for label, prob2: prob of missing synsets for triple")
        f.write("\n")
    # initilization of file   
    with open(write_file_freq, "w", newline='',encoding="UTF-8") as f:
        f.write("0,1,2 meaning the frequenct position of node_id. 0 meanns largest.-1 means no node_id")
        f.write("\n")
    
    for head, lines in load_file_chunk(read_file, n_each, n_total):
        cn_test = generate_gold_file(lines)
        
        label_embeddings = candidates_embeddings(cn_test, model_embedding)

        cn_predict,freq = sentence_embedding(cn_test, label_embeddings = label_embeddings)
        
        write_prediction(write_file, cn_predict)
        
        temp1, temp2 = no_synset_count(cn_predict)
        count1+=temp1
        count2+= temp2
        
        total_count += len(cn_predict)
        print("  prob1:", count1/(2*total_count), "prob2:" , count2/total_count)
        
        # write probility into file
        with open(write_file_prob, "a", newline='',encoding="UTF-8") as f:
            f.write(f"prob1: {count1/(2*total_count)}, prob2: {count2/total_count}, total_count: {total_count}")
            f.write("\n")
            
        # write freq distribution into file
        with open(write_file_freq, "a", newline='',encoding="UTF-8") as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerows(freq)

        
        # release RAM
        del cn_test, label_embeddings,temp1,temp2,cn_predict,f, lines, freq
        
        end = time.time()
        print("running time:", end- start)

 100 / 100  prob1: 0.31 prob2: 0.39
running time: 11.900064468383789
 100 / 100  prob1: 0.3125 prob2: 0.395
running time: 20.85469889640808
 100 / 100  prob1: 0.2816666666666667 prob2: 0.36333333333333334
running time: 30.516653537750244
  prob1: 0.2816666666666667 prob2: 0.36333333333333334
running time: 30.518649339675903
